In [ ]:
%%capture
#install required libraries
!pip install accelerate bitsandbytes tiktoken

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig, GenerationConfig, TextStreamer

from typing import Union, Tuple, List

In [ ]:
def instantiate_huggingface_model(
    model_name,
    quantization_config=None,
    device_map="auto",
    use_cache=True,
    trust_remote_code=None,
    pad_token=None,
    padding_side="left"
):
    """
    Instantiate a HuggingFace model with optional quantization using the BitsAndBytes library.

    Parameters:
    - model_name (str): The name of the model to load from HuggingFace's model hub.
    - quantization_config (BitsAndBytesConfig, optional): Configuration for model quantization.
      If None, defaults to a pre-defined quantization configuration for 4-bit quantization.
    - device_map (str, optional): Device placement strategy for model layers ('auto' by default).
    - use_cache (bool, optional): Whether to cache model outputs (False by default).
    - trust_remote_code (bool, optional): Whether to trust remote code for custom layers (True by default).
    - pad_token (str, optional): The pad token to be used by the tokenizer. If None, uses the EOS token.
    - padding_side (str, optional): The side on which to pad the sequences ('left' by default).

    Returns:
    - model (PreTrainedModel): The instantiated model ready for inference or fine-tuning.
    - tokenizer (PreTrainedTokenizer): The tokenizer associated with the model.

    The function will throw an exception if model loading fails.
    """

    # If quantization_config is not provided, use the default configuration
    if quantization_config is None:
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True,
            low_cpu_mem_usage=True,
        )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map=device_map,
        use_cache=use_cache,
        trust_remote_code=trust_remote_code
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name,
                                              trust_remote_code=trust_remote_code)

    if pad_token is not None:
        tokenizer.pad_token = pad_token
    else:
        tokenizer.pad_token = tokenizer.eos_token

    tokenizer.padding_side = padding_side

    return model, tokenizer

In [ ]:
model, tokenizer = instantiate_huggingface_model("Deci/DeciCoder-6B", trust_remote_code=True)

In [ ]:
streamer = TextStreamer(tokenizer)

deci_generator = pipeline("text-generation",
                          model=model,
                          tokenizer=tokenizer,
                          device_map="auto",
                          max_length=2048,
                          temperature=1e-3,
                          do_sample=True,
                          streamer=streamer
)

In [ ]:
def generate_text(pipeline, prompt):
    """
    Generates text using a specified pipeline based on a given prompt.

    This function takes a pipeline object, which is typically a pre-trained
    model loaded via the transformers library, and a prompt string. It then
    uses the pipeline to generate text based on the prompt.

    :param pipeline: A pre-initialized pipeline object for text generation.
    :param prompt: A string containing the prompt to guide text generation.
    :return: A string containing the generated text.
    """
    result = pipeline(prompt)[0]['generated_text']

# Prompt

In [ ]:
prompt = "def all_odd_elements(sequence):\n    \"\"\"Returns every odd element of the sequence.\"\"\""
generate_text(deci_generator, prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def all_odd_elements(sequence):
    """Returns every odd element of the sequence."""
    return [x for x in sequence if x % 2 != 0]<|endoftext|>


In [ ]:
prompt = "def nth_element_in_fibonnaci(element):\n    \"\"\"Returns the nth element of the Fibonnaci sequence.\"\"\""
generate_text(deci_generator, prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def nth_element_in_fibonnaci(element):
    """Returns the nth element of the Fibonnaci sequence."""
    if element == 0:
        return 0
    elif element == 1:
        return 1
    else:
        return nth_element_in_fibonnaci(element - 1) + nth_element_in_fibonnaci(element - 2)

def fibonacci_sequence(n):
    """Returns the Fibonacci sequence up to n elements."""
    sequence = []
    for i in range(n):
        sequence.append(nth_element_in_fibonnaci(i))
    return sequence

if __name__ == '__main__':
    n = int(input('Enter the number of elements: '))
    print(fibonacci_sequence(n))<|endoftext|>


# Instruct

In [ ]:
prompt = "Write code in Python to check if a number is prime"
generate_text(deci_generator, prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Write code in Python to check if a number is prime or not.

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

# Test
print(is_prime(7)) # True
print(is_prime(8)) # False<|endoftext|>


In [ ]:
prompt = "Write code in Java to check if a number is prime"
generate_text(deci_generator, prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Write code in Java to check if a number is prime or not.

public class PrimeNumber {
    public static void main(String[] args) {
        int n = 10;
        boolean isPrime = true;
        for (int i = 2; i < n; i++) {
            if (n % i == 0) {
                isPrime = false;
                break;
            }
        }
        if (isPrime) {
            System.out.println(n + " is a prime number");
        } else {
            System.out.println(n + " is not a prime number");
        }
    }
}<|endoftext|>


In [ ]:
prompt = "Write code in Rust to check if a number is prime"
generate_text(deci_generator, prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Write code in Rust to check if a number is prime or not.

fn is_prime(n: u32) -> bool {
    if n == 1 {
        return false;
    }
    for i in 2..n {
        if n % i == 0 {
            return false;
        }
    }
    true
}

fn main() {
    let n = 5;
    println!("Is {} prime? {}", n, is_prime(n));
}<|endoftext|>
